In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
import math
from micrograd.engine import Value
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
a = Value(-4.0)
b = Value(2.0)


In [0]:
a

In [0]:
b

In [0]:
c = a + b
d = a * b + b**3

In [0]:
c

In [0]:
d

In [0]:
c += c + 1
c += 1 + c + (-a)

In [0]:
c

In [0]:
d += d * 2 + (b + a).relu()
d += 3 * d + (b - a).relu()

In [0]:
d

In [0]:
e = c - d

In [0]:
e

In [0]:
f = e**2

In [0]:
f

In [0]:
g = f / 2.0
g += 10.0 / f

In [0]:
g

In [0]:
print(f'{g.data:.4f}') # prints 24.7041, the outcome of this forward pass

In [0]:
g.backward()
print(f'{a.grad:.4f}') # prints 138.8338, i.e. the numerical value of dg/da
print(f'{b.grad:.4f}') # prints 645.5773, i.e. the numerical value of dg/db

In [0]:
from micrograd import nn
n = nn.Neuron(2)
x = [Value(1.0), Value(-2.0)]
y = n(x)
#dot = draw_dot(y)

In [0]:
def f(x):
    return 3*x**2 - 4*x +5

In [0]:
f(3.0)

In [0]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
xs = np.arange(-5, 5, 0.25)
ys = f(xs)
plt.plot(xs, ys)

In [0]:
h = 0.00001
x = 3.0
(f(x + h) - f(x))/h

In [0]:
h = 0.0000001
x = -3.0
(f(x + h) - f(x))/h

In [0]:
h = 0.0000001
x =  2/3
(f(x + h) - f(x))/h

In [0]:
h = 0.0001
# inputs
a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
print (d)
c += h
d2 = a*b + c
print('d1', d1)
print('d2', d2)
print('slope', (d2-d1)/h)

In [0]:
class Value:
    
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0 # no effect
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward    
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out
    
    #only thing that matters is that we know how to create local derivative
    #functions can be simple or complex
    def tanh(self):
        n = self.data
        t = (math.exp(2*n) - 1)/(math.exp(2*n) + 1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
    
    def backward(self):

        # topological sort of children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


In [0]:
a = Value(2.0, label = 'a')
b = Value(-3.0, label = 'b')
c = Value(10.0, label = 'c')
e = a*b; e.label='e'
d = e+c; d.label='d'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L

In [0]:
print (a)
print (b)
print (c)
print (d)
print (d._prev)
print (d._op)

In [0]:
from graphviz import Digraph

In [0]:
def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges
    
def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            dot.node(name = uid + n._op, label = n._op)
            dot.edge(uid + n._op, uid)
        
    for n1, n2 in edges:
        #connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
        
    return dot

In [0]:
## Forward Pass

In [0]:
L.grad = 1.0
d.grad = -2.0
f.grad = 4.0
c.grad = -2.0
e.grad = -2.0

In [0]:
draw_dot(L)

In [0]:
# dL / dc
# dL / de
# we know L
#if we wiggle C, how does that impact L through D?
# dd / dc ?

# Derivative: (f(x+h) - f(x)) / h

(c + h + e) 

In [0]:
def lol():
    
    h = 0.001
    
    a = Value(2.0, label = 'a')
    b = Value(-3.0, label = 'b')
    c = Value(10.0, label = 'c')
    e = a*b; e.label='e'
    d = e+c; d.label='d'
    f = Value(-2.0, label='f')
    L = d * f; L.label = 'L'
    L1 = L.data
    
    a = Value(2.0, label = 'a')
    b = Value(-3.0, label = 'b')
    c = Value(10.0, label = 'c')
    c.data += h
    e = a*b; e.label='e'
    #e.data += h
    d = e+c; d.label='d'
    f = Value(-2.0, label='f')
    L = d * f; L.label = 'L'
    L2 = L.data
    
    print ((L2 - L1)/h)
    
lol()

In [0]:
plt.plot(np.arange(-5,5,0.2), np.tanh(np.arange(-5,5,0.2))); plt.grid();

In [0]:
# inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias of the neuron
b = Value(6.88137358, label='b')

# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1w1'
x2w2 = x2*w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1x2w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'


In [0]:
draw_dot(o)

In [0]:
o.backward()

In [0]:
o.grad = 1.0

In [0]:
o.grad = 1.0

# topological sort of children in the graph
topo = []
visited = set()
def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev:
            build_topo(child)
        topo.append(v)
build_topo(o)

for node in reversed(topo):
    node._backward()

In [0]:
o._backward()

In [0]:
n._backward()

In [0]:
b._backward()

In [0]:
x1w1x2w2._backward()

In [0]:
x2w2._backward()

In [0]:
x1w1._backward()

In [0]:
x1.grad = w1.data * x1w1.grad
w1.grad = x1.data * x1w1.grad

In [0]:
x2.grad = w2.data * x2w2.grad
w2.grad = x2.data * x2w2.grad

In [0]:
x2w2.grad = 0.5
x1w1.grad = 0.5

In [0]:
x1w1x2w2.grad = 0.5
b.grad = 0.5

In [0]:
n.grad = 0.5

In [0]:
o.grad = 1.0

In [0]:
1 - o.data**2

In [0]:
# o = tanh(n)
# d/dx tanxhx = 
# do/dn = 1 - tanh(n)**2
# do/dn = 1 - o**2

In [0]:
a = Value(3.0, label='a')
b = a + a; b.label = 'b'
b.backward()
draw_dot(b)

In [0]:
#dataiku.insights.save_figure(draw_dot(n))
client = dataiku.api_client()
project = client.get_default_project()

In [0]:
from dataiku import insights

In [0]:
my_fig = draw_dot(n)

In [0]:
insights.save_data?

In [0]:
insights.save_data("my_fig",my_fig,'text/plain')

In [0]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("mydataset")
mydataset_df = mydataset.get_dataframe()